In [ ]:
# -*- coding: utf-8 -*-
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Pandas (again)

URL https://github.com/FIIT-IAU

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
filename = 'data/sales-funnel.csv'
df = pd.read_csv(filename)
df.head()

## Descriptive statistics

In [ ]:
df.info()

In [ ]:
df.describe(include='all')

**How ​​many rows have a price greater than $8,000?**

In [ ]:
len(df[df.Price > 8000])

Since we are not interested in the rows themselves, but only in their number, the same result can be achieved more efficiently. By counting how many times the condition is met. Then there is no need to acquire data and for large datasets it can save a lot of time and resources.

In [ ]:
sum(df.Price > 8000)

Executing the condition returns a binary vector. True values ​​are considered as 1 during the sum operation, and False as 0. Thus, by summing, we get the number of rows where the condition was met.

**How ​​many lines have a status of "pending" and a price higher than $8,000?**

In [ ]:
sum((df.Status == 'pending') & (df.Price > 8000))

**How ​​many lines have a status of "pending" or a price higher than $8,000?**

In [ ]:
sum((df.Status == 'pending') | (df.Price > 8000))

**Create a column called "amount", which will be the product of the "price" and "quantity" columns.**
What is the sum of the values ​​in this column for rows with a status of "won"?

In [ ]:
df['amount'] = df.Price * df.Quantity
df.head()

In [ ]:
df[df.Status == 'won'].amount.sum()

**What is the total sum of the "amount" column for product records from the "CPU" category?**

In [ ]:
df[df.Product == 'CPU'].amount.sum()

# Changing the shape of the tables

## Pivot table

**What is the difference between the "pivot" and "pivot_table" functions?**

pivot_table allows you to define an aggregation function that combines values ​​for duplicate combinations in the index and column names.

**In the data set from the previous tasks, create a table where the rows will contain records for different products, the columns will contain records for different statuses, and the values ​​will be aggregated values ​​from the "amount" column.** Use the sum function from the numpy package for aggregation.

In [ ]:
pd.pivot_table(df, values='amount', index='Product', columns='Status', aggfunc=np.sum)

## Groupby

In [ ]:
df.groupby(['Product', 'Status']).amount.sum().unstack()

## Cross tab
**Create a crosstab that will show how many records are in individual states (status) for individual products**

In [ ]:
pd.crosstab(df.Product, df.Status)

**Create the table from the first task, but replace the products with Managers or Reps or Manager and Rep pairs. So you will have 3 different tables.**

In [ ]:
pd.crosstab(df.Manager, df.Status)

In [ ]:
pd.crosstab(df.Rep, df.Status)

In [ ]:
pd.crosstab([df.Manager, df.Rep], df.Status)

## Another dataset
### 1. Load the dataset - note: encoding "latin1"

In [ ]:
df2 = pd.read_csv('data/crunchbase_monthly_export.csv', encoding='latin1')
df2.head()

### 2. Descriptive statistics

This is a dataset of information about companies/projects that applied for investment and whether they were successful in the application

In [ ]:
df2.info()

The dataset has many columns containing strings even though we would logically expect a number there. It also contains quite a lot of missing values.

In [ ]:
df2.describe(include='all')

### 3. What is the maximum funding received? 
Hint: you will need to transform and adjust the values. Take a good look at the data type in the column and the column name.

In [ ]:
df2.columns

some column names have leading and trailing spaces

In [ ]:
df2[' funding_total_usd '].head(10)

the tracked column contains strings and several unwanted symbols that will need to be removed

In [ ]:
# df2[' funding_total_usd '] = pd.to_numeric(df2[' funding_total_usd '].str.replace(r'\D', ''))
df2[' funding_total_usd '] = pd.to_numeric(df2[' funding_total_usd '].str.replace(r'\D', '', regex=True))

**Caution**, I am doing a dangerous operation in the previous cell. I am changing the values ​​of the same column I am currently processing. 

**A more correct way of doing this would be to create a new column with a different name.** Because if this transformation does something unexpected, I won't keep the original copy of the data and won't be able to run it again.

### 4. Rename the columns so that they are understandable and properly formatted.

In [ ]:
df2 = df2.rename(columns=str.strip)
df2.columns

### 5. Create a table of values ​​from the "Market" column and the sum of the total funding values ​​for records with a given value in the "Market" column. Sort these values ​​from largest to smallest and display the first 10 of them.

In [ ]:
df2.groupby('market')\
    .funding_total_usd.sum()\
    .reset_index()\
    .sort_values(by='funding_total_usd', ascending=False)\
    .head(10)

### 6. How many companies are in the "Game" category?

In [ ]:
sum(df2.category_list.apply(lambda x: 'Game' in str(x).split('|')))

### 7. What is the average total funding for companies from New York? from San Francisco? Compare these values

In [ ]:
df2[df2.city == 'New York'].funding_total_usd.mean()

In [ ]:
df2[df2.city == 'San Francisco'].funding_total_usd.mean()